In [2]:
#!/usr/bin/env python
import numpy as np
import theano
T = theano.tensor
floatX = theano.config.floatX
import theano.tensor.nnet.conv3d2d

def random_matrix(shape, np_rng, name=None):
	return theano.shared(np.require(np_rng.randn(*shape), dtype=floatX),
			borrow=True, name=name)

# define inputs and filters
batchsize     = 16
in_channels   = 1
in_time       = 25
in_width      = 60
in_height     = 50
flt_channels  = 10
flt_time      = 2
flt_width     = 3
flt_height    = 4
rng = np.random.RandomState(42)
inputs = random_matrix((batchsize, in_time, in_channels, in_height, in_width),
		rng, 'inputs')
filters = random_matrix((flt_channels, flt_time, in_channels, flt_height, flt_width),
		rng, 'filters')
bias = random_matrix((flt_channels,),
		rng, 'bias')

# define first variant (conv3d2d)
convA = T.nnet.conv3d2d.conv3d(
	signals=inputs,  # Ns, Ts, C, Hs, Ws
	filters=filters, # Nf, Tf, C, Hf, Wf
	signals_shape=(batchsize, in_time, in_channels, in_height, in_width),
	filters_shape=(flt_channels, flt_time, in_channels, flt_height, flt_width),
	border_mode='valid')
convA = convA + bias.dimshuffle('x','x',0,'x','x')

# define second variant (conv3D), should be the same
filters_flip = filters[:,::-1,:,::-1,::-1]  # flip time, width and height
convB = T.nnet.conv3D(
	V=inputs.dimshuffle(0,3,4,1,2),  # (batch, row, column, time, in channel)
	W=filters_flip.dimshuffle(0,3,4,1,2), # (out_channel, row, column, time, in channel)
	b=bias,
	d=(1,1,1))
convB = convB.dimshuffle(0,3,4,1,2)  # (batchsize, time, channels, height, width)

# compile both
print "Compiling..."
funcA = theano.function([], convA)
funcB = theano.function([], convB)

# run both
print "Executing..."
A = funcA()
B = funcB()

# compare
print "A.shape =", A.shape
print "B.shape =", B.shape
print "allclose(A, B) =", np.allclose(A, B)
print A[0,0,0,0,:10]
print B[0,0,0,0,:10]



INFO (theano.gof.compilelock): Refreshing lock /Users/dan/.theano/compiledir_Darwin-14.4.0-x86_64-i386-64bit-i386-2.7.11-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /Users/dan/.theano/compiledir_Darwin-14.4.0-x86_64-i386-64bit-i386-2.7.11-64/lock_dir/lock


Compiling...


ImportError: ('The following error happened while compiling the node', Conv3D(InplaceDimShuffle{0,3,4,1,2}.0, InplaceDimShuffle{0,3,4,1,2}.0, bias, TensorConstant{(3,) of 1}), '\n', 'dlopen(/Users/dan/.theano/compiledir_Darwin-14.4.0-x86_64-i386-64bit-i386-2.7.11-64/tmp2PsVpV/fe8ce72d5cd6171fac8d0c68aa3e82e3.so, 2): Library not loaded: libmkl_intel_lp64.dylib\n  Referenced from: /Users/dan/.theano/compiledir_Darwin-14.4.0-x86_64-i386-64bit-i386-2.7.11-64/tmp2PsVpV/fe8ce72d5cd6171fac8d0c68aa3e82e3.so\n  Reason: image not found', '[Conv3D(<TensorType(float64, 5D)>, <TensorType(float64, 5D)>, bias, TensorConstant{(3,) of 1})]')